In [5]:
import pandas as pd
import json
import os

# Paths

In [6]:
save_path = "/data/AS/Epilepsy/SNOMED_FILTER_dicts"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    with open(os.path.join(save_path,"ReadME.txt"), "w") as text_file:
        text_file.write(f"These dictionaries were created from {os.getcwd()}")


# Import snomed tools

In [7]:
# open working with snomed tools
path = '/data/SNOMED/working_with_snomed/'
with open(os.path.join(path,"isa_rela_pt2ch_20200401.json")) as json_file:
    pt2ch = json.load(json_file)
with open(os.path.join(path,"isa_rela_ch2pt_20200401.json")) as json_file:
    ch2pt = json.load(json_file)
with open(os.path.join(path,"SCTID2name.json")) as json_file:
    SCTID2name = json.load(json_file)


def get_all_children(snomed): 
    """Get all children function"""
    result = [] 
    stack = [snomed] 
    while len(stack) != 0: 
        # remove the last element from the stack
        current_snomed = stack.pop()
        current_snomed_parent = pt2ch.get(current_snomed, [])
        stack.extend(current_snomed_parent)
        result.append(current_snomed)
        result = list(set(result))
    return result


# Epilepsy classification

In [8]:
# All children concepts
epilepsy_children = []
for child in get_all_children("S-84757009"):
    epilepsy_children.append(SCTID2name[child][0])
print(len(epilepsy_children))
for sctid in epilepsy_children:
    print(sctid)


215
Infantile convulsion and choreoathetosis syndrome (disorder)
Generalized convulsive epilepsy (disorder)
Epilepsy (disorder)
Rolandic epilepsy, speech dyspraxia syndrome (disorder)
Refractory frontal lobe epilepsy (disorder)
Alcohol-induced epilepsy (disorder)
Post-traumatic epilepsy (disorder)
Epileptic encephalopathy (disorder)
Benign non-familial neonatal convulsions (disorder)
Early-onset Lafora body disease (disorder)
Refractory juvenile myoclonic epilepsy (disorder)
Lateral temporal epilepsy (disorder)
Refractory myoclonic epilepsy (disorder)
Primary generalized absence epilepsy (disorder)
Infantile spasms co-occurrent with status epilepticus (disorder)
Pyridoxine-dependent epilepsy (disorder)
Status epilepticus (disorder)
Epilepsy due to perinatal anoxic-ischemic brain injury (disorder)
Visual epilepsy (disorder)
X-linked spasticity, intellectual disability, epilepsy syndrome (disorder)
Epilepsy, microcephaly, skeletal dysplasia syndrome (disorder)
Epilepsy co-occurrent and d

In [9]:
relevant_diagnosis = {
    "Epilepsy": ["84757009"], # Epilepsy (disorder)
}

In [10]:
final_relevant_diagnosis = {}
for k, v in relevant_diagnosis.items():
    children = []
    for item in v:
        code = "S-"+str(item) # remove in new version of medcat
        try:
            SCTID2name[code]
            children.extend(get_all_children(code))
        except KeyError:
            print(f"SCTID {code} does not exisit in your current snomed version")
        children = list(set(children))
    final_relevant_diagnosis[k] = children
    

In [11]:
json.dump(final_relevant_diagnosis, open(os.path.join(save_path,"epilepsy_diagnosis.json"), "w"))
del final_relevant_diagnosis

# Seizure freedom outcome

In [27]:
seizure_outcomes = {
    "seizure" : ["91175000"],  #  | Seizure (finding) |
    "dissociative convulsions": ["191714002"], # |Dissociative convulsions (dosorder) |
    "seizure free" : ["370994008"],  # | Seizure free (finding) |
}


In [28]:
final_seizure_outcomes = {}
for k, v in seizure_outcomes.items():
    children = []
    for item in v:
        code = "S-"+str(item) # remove in new version of medcat
        try:
            SCTID2name[code]
            children.extend(get_all_children(code))
        except KeyError:
            print(f"SCTID {code} does not exisit in your current snomed version")
        children = list(set(children))
    final_seizure_outcomes[k] = children
    

In [29]:
# Subtract epilepsy from seizure
final_seizure_outcomes['seizure'] = [sz_term for sz_term in final_seizure_outcomes['seizure'] if sz_term not in get_all_children("S-84757009")]

In [30]:
# Subtract Non epileptic from seizures
final_seizure_outcomes['seizure'].remove("S-864171000000103")

In [34]:
# add Non epileptic to dissociative
final_seizure_outcomes['dissociative convulsions'].append("S-864171000000103")

In [36]:
json.dump(final_seizure_outcomes, open(os.path.join(save_path,"seizure_freedom_outcomes.json"), "w"))
del final_seizure_outcomes

# Dictionary of other relevant symptoms

In [39]:
relevant_symptoms ={
    "pregnant": ["77386006"],
    "breastfeeding mother": ["413712001"],
    "unrinary incontinence": ["165232002"],
    #Mental Health
    "anxiety": ["48694002"],
    "depression": ["35489007"],
    "lethargy": ["248274002"],
    "headache": ["25064002"],
    #Side effects
    "dizziness": ["404640003"],
    "rash": ["271807003"],
    "nausea": ["422587007"],
    #Comorbidities
    "diabetes": ["73211009"],
    "hypertension": ["38341003"],
    "myocardial infarction": ["22298006"],
    "cerebrovasular accident": ["230690007"],
    "asthma" : ['195967001'],
    "chronic obstructive lung disorder": ["13645005"],
    "fibromyalgia": ["203082005"],
    "dementia": ["52448006"],
}

mental health
symptom that are side effects dizziness nausea etc
Other comorbid diseases
Comorbid high level groups
Look at lancet epide.. comorbidities
Hypertension, MI
cerebrovas
diabetes etc


Other treatments

In [40]:
final_relevant_symptoms = {}
for k, v in relevant_symptoms.items():
    children = []
    for item in v:
        code = "S-"+str(item) # remove in new version of medcat
        try:
            SCTID2name[code]
            children.extend(get_all_children(code))
        except KeyError:
            print(f"SCTID {code} does not exisit in your current snomed version")
        children = list(set(children))
    final_relevant_symptoms[k] = children
    

In [41]:
json.dump(final_relevant_symptoms, open(os.path.join(save_path,"relevant_symptoms.json"), "w"))
del final_relevant_symptoms

# Dictionary of relevant therapies

In [120]:
relevant_therapy = {
    "Ketogenic diet" : ["765060000"], # Ketogenic diet (regime/therapy)
    "Vagal nerve stimulator in situ": ["700210008"],
    # Add dbs here or stimulation
}

In [121]:
final_relevant_therapy = {}
for k, v in relevant_therapy.items():
    children = []
    for item in v:
        code = "S-"+str(item) # remove in new version of medcat
        try:
            SCTID2name[code]
            children.extend(get_all_children(code))
        except KeyError:
            print(f"SCTID {code} does not exisit in your current snomed version")
        children = list(set(children))
    final_relevant_therapy[k] = children
    

In [122]:
json.dump(final_relevant_therapy, open(os.path.join(save_path, "relevant_epilepsy_therapies.json"), "w"))
del final_relevant_therapy

# AED

In [123]:
aed_dict = {"Acetazolamide" : ["372709008", # | Acetazolamide (substance) |
                "33664007", # | Product containing acetazolamide (medicinal product) |
                 "9260001000001101", # | Diamox (product) |
               ],
"Brivaracetam" : ["420813001", # | Brivaracetam (substance) |
              "421052000", # | Product containing brivaracetam (medicinal product) |
                "12643501000001103", # | Briviact (product) |
              ],
"Cannabidiol" : ["96223000", # | Cannabidiol (substance) |
              "766895006", # | Product containing cannabinoid (product) |
               "114881000001100", # | Epidyolex (product) |
             ],
"Carbamazepine" : ["387222003", # | Carbamazepine (substance) |
                "40820003", # | Product containing carbamazepine (medicinal product) |
                "9361901000001100", # | Tegretol (product) |
               ],
"Clobazam" : ["387572002", # | Clobazam (substance) |
           "96236000", # | Product containing clobazam (medicinal product) |
            "9522301000001103", # | Frisium (product) |
            "12468101000001109", # | Perizam (product) |
            "11479301000001105", # | Tapclob (product) |
            "13033401000001102", # | Zacco (product) |
          ],
"Clonazepam" : ["387383007", # | Clonazepam (substance) |
            "69708003", # | Product containing clonazepam (medicinal product) |
            ],
"Eslicarbazepine" : ["697998007", # | Eslicarbazepine (substance) |
                   "715615002", # | Product containing eslicarbazepine (medicinal product) |
                  "700015007", # | Eslicarbazepine acetate (substance) |
                   "10368901000001108",# | Zebinix (product) |
                         ],
"Ethosuximide" : ["387244008", # | Ethosuximide (substance) |
              "44790008", # | Product containing ethosuximide (medicinal product) |
              ],

"Everolimus" : ["428698007", # | Everolimus (substance) |
            "428127005", # | Product containing everolimus (medicinal product) |
              "10365601000001108", # | Afinitor (product) |
              "12466401000001104", # | Certican (product) |
              "11079101000001101", # | Votubia (product) |
            ],
"Gabapentin" : ["386845007", # | Gabapentin (substance) |
             "108402001", # | Product containing gabapentin (medicinal product) |
              "9461401000001102", # | Neurontin (product) |
            ],
"Lacosamide" : ["441647003", # | Lacosamide (substance) |
            "441532003", # | Product containing lacosamide (medicinal product) |
              "9896001000001108", # | Vimpat (product) |
            ],
"Lamotrigine" : ["387562000", # | Lamotrigine (substance) |
             "96195007", # | Product containing lamotrigine (medicinal product) |
               "9474101000001100", # | Lamictal (product) |
             ],
"Levetiracetam" : ["387000003", # | Levetiracetam (substance) |
               "116076009", # | Product containing levetiracetam (medicinal product) |
                 "9452601000001103", # | Keppra (product) |
                 "11478701000001104", # | Desitrend (product) |
               ],
"Oxcarbazepine" : ["387025007", # | Oxcarbazepine (substance) |
               "116511002", # | Product containing oxcarbazepine (medicinal product) |
                 "9372701000001101", # | Trileptal (product) |
               ],
"Perampanel" : ["703127006", # | Perampanel (substance) |
            "703728005", # | Product containing perampanel (medicinal product) |
              "11468901000001106", # | Fycompa (product) |
            ],
"Phenobarbital" : ["373505007", # | Phenobarbital (substance) |
               "51073002", # | Product containing phenobarbital (medicinal product) |
               ],
"Phenytoin" : ["387220006", # | Phenytoin (substance) |
           "40556005", # | Product containing phenytoin (medicinal product) |
             "9903301000001108", # | Epanutin (Phenytoin) (product) |
           ],
"Piracetam" : ["395833008", # | Piracetam (substance) |
           "322998004", # | Product containing piracetam (medicinal product) |
             "9506301000001108", # | Nootropil (product) |
           ],
"Pregabalin" : ["415160008", # | Pregabalin (substance) |
            "415159003", # | Product containing pregabalin (medicinal product) |
            "12458001000001104", # | Alzain (product) |
              "12881801000001102", # | Axalid (product) |
              "12455901000001107", # | Lecaent (product) |
              "9513701000001102", # | Lyrica (product) |
            ],
"Primidone" : ["387256009", # | Primidone (substance) |
           "47120002", # | Product containing primidone (medicinal product) |
           ],
"Rufinamide" : ["429835003", # | Rufinamide (substance) |
            "429802008", # | Product containing rufinamide (medicinal product) |
              "9687001000001101", # | Inovelon (product) |
            ],
"Stiripentol" : ["428221002", # | Stiripentol (substance) |
             "427946009", # | Product containing stiripentol (medicinal product) |
               "9562601000001102", # | Diacomit (product) |
             ],
"Tiagabine" : ["372485004", # | Tiagabine (substance) |
           "108403006", # | Product containing tiagabine (medicinal product) |
             "9522801000001108", # | Gabitril (product) |
           ],
"Topiramate" : ["386844006", # | Topiramate (substance) |
            "108400009", # | Product containing topiramate (medicinal product) |
              "9369101000001104", # | Topamax (product) |
            ],
"Sodium_valproate" : ["387481005", # | Valproate sodium (substance) |
                  "10049011000001109", # | Sodium valproate (product) |
                    "231031000001104", # | Dyzantil (product) |
                    "9277801000001102", # | Epilim (product) |
                    "9277901000001108", # | Epilim Chrono (product) |
                    "9741001000001107", # | Epilim Chronosphere MR (product) |
                    "9278301000001108", # | Episenta (product) |
                    "9278401000001100", # | Epival CR (product) |
                  ],
"Valproic_acid" : ["387080000", # | Valproic acid (substance) |
               "13965000", # | Product containing valproic acid (medicinal product) |
                 "9246501000001103", # | Convulex (product) |
                 "231031000001104", # | Dyzantil (product) |
                 "9277901000001108", # | Epilim Chrono (product) |
               ],
"Valproate_semisodium" : ["9256001000001109", # | Depakote (product) |
                        "92271000001100", # | Syonell (product) |
                       ],
"Valproate" : ["264325000" # | Valproate (substance) | contains valproic acid and NA valporate
            ] + Valproate_semisodium + Sodium_valproate + Valproic_acid,
"Vigabatrin" : ["310283001", # | Vigabatrin (substance) |
             "322976008", # | Product containing vigabatrin (medicinal product) |
              "225771000001104", # | Kigabeq (product) |
              "9332401000001106", # | Sabril (product) |
             ],
"Zonisamide" : ["125693002", # | Zonisamide (substance) |
            "398762003", # | Product containing zonisamide (medicinal product) |
              "233091000001104", # | Desizon (product) |
              "9402301000001104", # | Zonegran (product) |
            ],
}

In [124]:
final_aed_dict = {}
for k, v in aed_dict.items():
    children = []
    for item in v:
        code = "S-"+str(item) # remove in new version of medcat
        try:
            SCTID2name[code]
            children.extend(get_all_children(code))
        except KeyError:
            print(f"SCTID {code} does not exisit in your current snomed version")
        children = list(set(children))
    final_aed_dict[k] = children


SCTID S-231031000001104 does not exisit in your current snomed version
SCTID S-231031000001104 does not exisit in your current snomed version
SCTID S-231031000001104 does not exisit in your current snomed version
SCTID S-231031000001104 does not exisit in your current snomed version
SCTID S-225771000001104 does not exisit in your current snomed version
SCTID S-233091000001104 does not exisit in your current snomed version


In [125]:
json.dump(final_aed_dict, open(os.path.join(save_path,"aed_dict.json"), "w"))
del final_aed_dict

In [126]:
# Other  Anticonvulsant (substance)

other_anticonvulsants = {"Desmethylparamethadione" : ["117154007"],
"Potassium_bromide" : ["35903003"],
"Sulthiamine" : ["50580004",
               "420672000",# | Product containing sulthiamine (medicinal product) |
              ],
"Sodium_bromide" : ["58693000"],
"Felbamate" : ["96194006",
            "371387009",# | Product containing felbamate (medicinal product) |
            ],
"Dimethadione" : ["96198009"],
"Barbiturate_antiepileptic" : ["372618002"],
"Benzodiazepine" : ["372664007"], # double check and remove already targetted subterms
"Oxazolidinedione" : ["372731000",
                    "41399007", #| Product containing oxazolidinedione (product) |
                   ],
"Succinimide" : ["372837008",
               "74632002" # | Product containing succinimide (product) |
              ],

"Paraldehyde" : ["387190006",
               "35324004", # | Product containing paraldehyde (medicinal product) |
              ],
"Beclamide" : ["391815006"],
"Piracetam" : ["395833008",
             "322998004" # | Product containing piracetam (medicinal product) |
            ],
"Clomethiazole" : ["395978007",
                 "354039005" # | Product containing clomethiazole (medicinal product) |
                ],
"Caramiphen" : ["412472003",
              "412473008" # | Product containing caramiphen (medicinal product) |
             ],
"Phenacemide" : ["18712002",
               "400358006" # | Product containing phenacemide (medicinal product) |
              ],
"Magnesium_sulfate" : ["387202002"],
"Hydantoin_derivative_anticonvulsant" : ["439255000"],
"Retigabine" : ["699271005",
              "704476003" # | Product containing retigabine (medicinal product) |
             ],
"Perampanel" : ["703127006"],
"Pheneturide" : ["725698000"],
"Tiagabine_hydrochloride" : ["108404000"],
"Tiagabine_hydrochloride_monohydrate" : ["725652001"],
"Gabapentin_enacarbil" : ["703122000"],
"Valpromide" : ["698808002"],
}

In [127]:
final_other_anticonvulsants_dict = {}
for k, v in other_anticonvulsants.items():
    children = []
    for item in v:
        code = "S-"+str(item) # remove in new version of medcat
        try:
            SCTID2name[code]
            children.extend(get_all_children(code))
        except KeyError:
            print(f"SCTID {code} does not exisit in your current snomed version")
        children = list(set(children))
    final_other_anticonvulsants_dict[k] = children
    
    

In [128]:
json.dump(final_other_anticonvulsants_dict, open(os.path.join(save_path,"other_anticonvulsants_dict.json"), "w"))
del final_other_anticonvulsants_dict

In [24]:
drugs = []
for term in Lamotrigine:
    code = "S-"+str(term)
    drugs.extend(get_all_children(code))
for drug in drugs:
    print(SCTID2name[drug][0])

Lamotrigine (substance)
Lamotrigine 5mg dispersible tablets (Arrow Generics Ltd) (product)
Lamotrigine 5mg/5ml oral solution (product)
Lamotrigine 50mg/5ml oral suspension (product)
Lamotrigine 5.5mg/5ml oral suspension (product)
Lamotrigine 25mg dispersible tablets (Dexcel Pharma Ltd) (product)
Lamotrigine 25mg dispersible tablets sugar free (Phoenix Healthcare Distribution Ltd) (product)
Lamotrigine 200mg tablets (Sigma Pharmaceuticals Plc) (product)
Lamotrigine 25mg tablets (Tillomed Laboratories Ltd) (product)
Lamotrigine 25mg dispersible tablets (Actavis UK Ltd) (product)
Lamotrigine 15mg/5ml oral solution (product)
Product containing precisely lamotrigine 200 milligram/1 each conventional release oral tablet (clinical drug)
Lamictal 200mg tablets (GlaxoSmithKline) (product)
Lamotrigine 100mg dispersible tablets sugar free (Sigma Pharmaceuticals Plc) (product)
Lamictal 50mg tablets (PI) (Waymade Ltd) (product)
Lamotrigine 100mg dispersible tablets sugar free (PI) (Sigma Pharmaceut

In [1]:
get_all_children("84757009")

NameError: name 'get_all_children' is not defined